In [1]:
import numpy as np
import pandas as pd
import re,math
import concurrent.futures
import multiprocessing
import threading
import logging
import time

# 设置日志级别
logging.basicConfig(level=logging.DEBUG)

# 设置函数运行计时用的修饰器
def timeit(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} took {end - start:.6f} seconds')
        return result
    return wrapper

# 高相似多少
coum=200
# 加载保存的数组
lin = np.load("lin.npy")
# lin = np.linspace(0,3,401)
# lin = np.linspace(0.01,1,100)


data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'583drugs39cell Zscore行标准化 synergy.xlsx')
drug_names = list(data1.index)

# 合并drugA和drugB为一列作为索引
# 顺序
temp1 = [] # 分子指纹合并值 【【】，【】，【】，】
index_1 = [] # 对应的名字列表 【【】，【】，【】，】
# 逆序
temp2 = []
index_2 = []
for index,row in data2.iterrows():
    ct = list((np.array(data1.loc[row["DrugA"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugB"],:]))))
    temp1.append(ct)
    index_1.append((row["DrugA"],row["DrugB"]))
    ct = list((np.array(data1.loc[row["DrugB"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugA"],:]))))
    temp2.append(ct)
    index_2.append((row["DrugB"],row["DrugA"]))

/root/miniconda3/envs/jupyter-env/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [2]:
# 两组数据的tanimoto计算
def tanimoto(p,q):
    tep1=0
    tep2=0
    lenthp=int(len(p))
    for i in range(lenthp):
        a=p[i]
        b=q[i]
        if (a==1)|(b==1): # 并
            tep1=tep1+1
        if (a==1)&(b==1): # 交
            tep2=tep2+1
    c=round((tep2 / tep1),4)      #取值4位数
    return c

def compute_tanimoto(i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx):
    tmp=[]
    a = np.array(i) #取出一分子指纹 转换列表为向量
    b = np.array(j)
    for k,m,k_n,m_n in zip(temp1,temp2,index_1,index_2): # 遍历每一个药物对的分子指纹
        c = np.array(k)
        d = np.array(m)
        tp1 = tanimoto(a,c)
        tp2 = tanimoto(a,d)
        tp3 = tanimoto(b,c)
        tp4 = tanimoto(b,d)
        tps = [tp1, tp2, tp3, tp4]
        index_list = [(i_n,k_n,tp1),(i_n,m_n,tp2),(j_n,k_n,tp3),(j_n,m_n,tp4)]
        index_location = tps.index(max(tps))
        tmp.append(max(tps))
        Tanimoto_index_dist[f"{i_n},{k_n}"] = index_list[index_location]
    return idx, tmp

temp = [None] * len(temp1)
Tanimoto_index_dist = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 提交任务时使用 enumerate ，并将索引值作为参数传递给任务函数
    futures = [executor.submit(compute_tanimoto, i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx)
               for idx, (i,j,i_n,j_n) in enumerate(zip(temp1,temp2,index_1,index_2))]
    for future in concurrent.futures.as_completed(futures):
        idx, result = future.result()
        temp[idx] = result
Tanimoto = pd.DataFrame(temp, columns=index_1, index=index_1)

In [3]:
# 获取columns数组 - 处理后的
ttpp = []
for i in list(Tanimoto.columns):
    ttpp.append(str(i)[1:-1]) # 去除首尾括号的药物对名字
data2 = pd.DataFrame(data2.iloc[:,2:].T.values, columns=list(ttpp)) # 药物对协同得分
columns = data2.columns # 修改后的药物对名字

In [4]:
# 删去对应行后的字典
dict_lines = {} # {键1：值1} 谷本系数字典
count = 0
ct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,ct]) # 每一列谷本系数 [:,1] 第一列的每一行 实际就是第一列
    y.pop(count)
    count += 1
    dict_lines[columns[ct]] = y # 谷本系数药物对的名字做键1，y是值1
    ct += 1

In [5]:
# 分离x y数据构造字典
dict_columns_split = {} # 高相似字典
# 计算x数据
x = data2.drop(columns=columns)  # 其他所有的列
for i in columns:
    # b 要预测协同得分那一列
    y = data2.loc[:, i]
    # A 权重
    x_i = x.copy()
    x_i['y'] = y

    # 高相似的    取高相似的药物对名字max1和对应的值max5
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_lines[i], index = tp)
    df.sort_values(by=0, inplace=True, ascending=True)
    max_l = df.index[-coum:]
    max_5x = pd.DataFrame([data2.loc[:, j] for j in max_l]).T

    # 存储 对应数据
    count = 0 # 第几行 int
    for j in list(y.index): # 35个索引 0~34 数字（字符串-str） 【0，1，2，3.。。。】
        dict_columns_split[f'{i},{j}']={} # 空字典 键-空字典
        dict_columns_split[f'{i},{j}']['y_line']=y[count]
        dict_columns_split[f'{i},{j}']['max_x_line']=max_5x.iloc[count,:].values
        count += 1

In [6]:
dict_columns_split["'SUNITINIB', 'MK-4827',32"]['max_x_line']

array([ 9.0132660e-02, -9.8969820e-02, -2.2030113e+00, -2.6533191e+00,
       -1.3656538e+00, -2.5928310e+00, -4.1963280e-02, -6.7495495e-01,
       -1.0962662e+00, -3.2805324e-01, -2.4593820e-01, -5.4777384e-01,
       -3.4914732e-01, -2.6289720e+00, -3.5352638e-01, -6.3626295e-01,
        2.5722393e-01,  2.7945230e-01,  1.1201497e+00, -1.2683841e+00,
       -1.5246437e+00, -8.9764000e-04, -1.2019012e+00, -1.7924271e+00,
        1.0922955e+00, -9.8660110e-01,  5.5708370e-01,  4.5372880e-01,
       -3.1579345e-01, -9.3183120e-01,  2.1274490e-01, -4.9613390e-01,
       -5.2834000e-01, -7.5484276e-01, -1.7530004e+00,  7.2603956e-02,
       -3.8278688e-02, -2.2323730e+00, -1.5543456e+00, -2.2530637e+00,
       -3.7979860e-01, -1.1786811e+00,  3.6453614e-01, -2.6384280e-01,
        2.0963123e-01,  1.5155060e-01, -6.2854420e-01,  4.4618350e-01,
       -6.8151623e-01,  8.6943164e-02,  1.2990282e+00,  1.8029979e-01,
       -9.0274096e-01,  2.7012500e-04, -1.7904884e+00, -4.8966548e-01,
      

In [7]:
TTT = [] # 值列表
cct = 0
for i in Tanimoto.index: 
    y = list(Tanimoto.iloc[:,cct])
    y.pop(cct) 
    tp_columns = list(columns.copy())
    tp_columns.pop(cct)
    df = pd.DataFrame(y, index=tp_columns)
    df.sort_values(by=0 , inplace=True, ascending=True) 
    max_l = df.index[-coum:]  
    # 计算max_5y
    max_5y = []
    for j in max_l:
        max_5y.append(Tanimoto.iloc[list(columns).index(j), cct])
    # 将max_5y加入TTT
    TTT += max_5y
    # 修改cct变量的值
    cct += 1

In [17]:
def alpha_pcc(alp, t_coum, a, linename, dict_columns_split, TTT, coum, data2):
    alpha = []
    result_pcc = []
    pcc_split = []
    pre = []
    for elem in range(39):
        pp = []
        for j in TTT[(coum * a - t_coum):(coum * a)]: 
            pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))#改函数
        pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
    pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
    result_pcc.append(np.max(pcc_split))
    pcc=result_pcc[0]
    del result_pcc, pcc_split, pre, pp
    return pcc

def alp_mse_rmse(alp, t_coum, a, linename, dict_columns_split, TTT, coum):
    r1 = []
    ct = 0
    for elem in range(39):
        pp = []
        for j in TTT[(coum * a - t_coum):(coum * a)]:
            pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  # 改函数
        r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) -
                        dict_columns_split[f"{linename},{elem}"]['y_line'])
    mse=np.mean(np.square(r1))
    rmse=math.sqrt(mse)
    return mse, rmse

In [18]:
# 单进程 分函数
def calculate_parameters(alp, t_coum, coum, TTT, data2, dict_columns_split):
    a = 1
    Name_data = []
    for linename in columns:
        # 求PCC
        pcc = alpha_pcc(alp, t_coum, a, linename, dict_columns_split, TTT, coum, data2)
        # 求MSE
        mse, rmse = alp_mse_rmse(alp, t_coum, a, linename, dict_columns_split, TTT, coum)
        Name_data.append([linename, t_coum, alp, mse, pcc, rmse])
        a = a+1
    pcc_mean = pd.DataFrame(Name_data, columns=['drug', 'coum', "alp", "mse", "pcc", "rmse"])["pcc"].mean()
    print(pcc_mean, alp)
    return [pcc_mean, alp, Name_data]

In [ ]:
# 多进程
@timeit
def run_parallel(columns, lin):
    with multiprocessing.Pool() as pool:
        tp = []
        for alp in lin:
            for t_coum in range(2, coum+1, 1):    #邻居
                tp.append((alp, t_coum, coum, TTT, data2, dict_columns_split,))
        results = pool.starmap(calculate_parameters, tp)
    return results

PCC_mean = 0
res_pcc_list = []
results = run_parallel(columns,lin)
sorted_list = sorted(results, key=lambda x: x[0])
max_res = sorted_list[-1][2]
print("MAX PCC: ", sorted_list[-1][0])
print("ALP:", sorted_list[-1][1])
# 将结果保存到 CSV 文件中
pd.DataFrame(max_res).to_csv('pcc_cpu_test_multi_unite_alp.csv', header=None, index=False)

0.31403596652673277 1.0
0.34241853667438493 1.0
0.3742804329280965 1.0
0.4065966624993245 1.0
0.5768039829008392 1.015
0.4269636917850709 1.0
0.44379223225657577 1.0
0.6112139375572742 1.03
0.5460236065707919 1.125
0.5789169866058282 1.015
0.4552562183636186 1.0
0.5494204967225623 1.125
0.46626341186601555 1.0
0.46623418687457047 1.235
0.5639677086951811 1.045
0.5804112211931726 1.015
0.608225494379816 1.1400000000000001
0.4996436316927354 1.06
0.4811778075670963 1.235
0.4811927748137632 1.0
0.5536862907262426 1.125
0.6104449412785024 1.03
0.5870410879942719 1.155
0.5938983216664462 1.25
0.49091485862951245 1.235
0.49092424546022206 1.0
0.58316629996507 1.015
0.4996724748842267 1.235
0.5572406829993629 1.125
0.6083461376592951 1.1400000000000001
0.5215078126583002 1.17
0.44840201489222964 0.6040043876104081.075
 1.2650000000000001
0.49968181589422644 1.0
0.5695664157460714 1.3599999999999999
0.5853854944460302 1.015
0.506165179109859 1.235
0.5602387198094438 1.125
0.4649751126300935 1.

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



 1.5350000000000001
0.5415095999969927 1.865
0.503457450613765 1.77
0.5572500343308867 1.6
0.4053513524585228 1.675
0.43422314637710796 1.925
0.609826982707213 1.88
0.38347930375273354 1.815
0.5460355020647647 1.865
0.43612439897086247 1.7850000000000001
0.5130406765954845 1.645
0.6108150368043572 1.755
0.5602465348753953 1.6
0.3723550371350818 1.69
0.3985914630877811 1.8
0.5494297607540916 1.865
0.4700556935719729 1.6600000000000001
0.5831833275447867 0.36369805436142961.63 
1.7349999999999999
0.5854053811764605 1.615
0.5639112309511929 1.6
0.3804995685094618 1.97
0.608540801792219 1.88
0.36912499843956004 1.705
0.5536982323741672 1.865
0.5444707188851211 1.505
0.6105138144524686 1.755
0.45924616441503824 1.52
0.39036108779014284 1.955
0.5672446636739735 1.6
0.4448816208931953 1.55
0.38327620755151864 1.565
0.4045241280070982 1.94
0.3652278798005334 1.845
0.5008412295008107 1.77
0.5572520751801275 1.865
0.5217653217332391 1.895
0.44777812418564616 1.9100000000000001
0.3713595936110509

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)




0.3652761923803077 1.83
0.5339816299541827 1.695
0.46832979132986136 1.645
0.3805380021799804 1.955
0.4190494229688482 1.925
0.6053758346563429 1.85
0.6111181499313868 1.6
0.5466685190904921 1.88
0.5940896166613376 1.615
0.5650490750399848 1.755
0.3855022971949251 1.94
0.58540657392987 1.71
0.6098282752373559 1.865
0.6105144849905159 1.74
0.600082132263668 1.585
0.43652422141580227 1.6600000000000001
0.5392076270840701 1.695
0.47315869904058 1.895
0.42886166358500744 1.52
0.6064732853607468 1.85
0.38637062487074414 1.675
0.41902548774979614 1.55
0.6075325177118913 1.63
0.36932009675240135 1.9849999999999999
0.6085423922061385 1.865
0.5922100205110905 1.615
0.5415087619155852 1.695
0.3675226378014617 1.565
0.41813124764643766 1.7850000000000001
0.41487301656584796 1.53500000000000010.562389394953471 1.755

0.5890080083503809 1.71
0.5439000705084783 1.88
0.3637563554289103 1.72
0.607649309199313 1.85
0.4177838817369149 1.9100000000000001
0.46636820310517757 1.645
0.6011895958616942 1.58

...

0.5880660300755459 2.6799999999999997
0.4821331545814401 2.975
0.5734349884725471 2.635
0.3891593947483388 0.6103512282285671 2.65000000000000042.725

0.36933525611410556 2.74
0.5973968664574264 2.8049999999999997
0.5596442957376445 2.5250000000000004
0.412319244552597 2.71
0.586142284673195 2.665
0.3724650152200332 2.755
0.5071825163868798 2.96
0.4701370134347108 2.6950000000000003
0.48868612997188554 2.5549999999999997
0.37943685881700767 2.6
0.5768342694316493 2.635
0.5212045372669286 2.51
0.40346504712747133 2.585
0.6110696585354747 2.7750000000000004
0.5409704969961076 2.54
0.611577194332849 2.6500000000000004
0.45914254797998005 2.5700000000000003
0.5789455618649488 0.3694640974806464 2.6352.615

0.5861399222405325 2.6799999999999997
0.36154184791313393 2.925
0.5954542174295696 2.8049999999999997
0.55649827065757 2.5250000000000004
0.5804311528160199 2.635
0.610260157334641 2.7750000000000004
0.37342729625400317 2.91
0.48041790284230695 2.975
0.6107997239923916 2.6500000000000004

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)




0.4064785990402612 2.76
0.5695783064959924 2.62
0.43324956211214893 2.5700000000000003
0.42688349209353227 2.76
0.5494320992744994 2.915
0.6037089440250064 2.79
0.4437269852924864 2.76
0.5728586525124533 2.62
0.5537014634022531 2.915
0.5515125375744241 2.7750000000000004
0.45518609010810895 2.76
0.4295711449907853 2.6950000000000003
0.6083474268183391 2.745
0.48612207837023025 2.99
0.5902512460605633 2.635
0.4661873748591061 2.76
0.5094886308561883 2.8049999999999997
0.5734349631028413 2.62
0.557255104388787 2.915
0.5409232208986843 2.6500000000000004
0.46833019237329265 2.96
0.48115261153792455 2.76
0.43293342760552617 2.975
0.49768586344553145 2.5250000000000004
0.5267784849405576 2.665
0.6022635825432532 2.79
0.4032346726569981 2.835
0.4908980814003905 0.51410149796710462.76 
2.9450000000000003
0.5602505277200817 2.915
0.5768342190196915 0.60906316629771652.62 2.745


0.37124800139175906 2.880.43930857586662186 2.5549999999999997
0.6005311369132615 2.9299999999999997
0.499655618136

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)




0.5639174407971895 2.98
0.46836339195572096 2.9000000000000004
0.39799853347869163 2.6799999999999997
0.40761718009278003 2.7750000000000004
0.5161057366482716 2.715
0.5987116921800478 2.56
0.5938052953410401 2.84
0.5339515877685354 2.87
0.5672540332659373 2.98
0.3829744203856631 2.8049999999999997
0.6103515501129912 2.575
0.6001016195757418 2.56
0.4512967826424939 2.76
0.4628422498267113 2.855
0.5695791985418582 2.98
0.388943423269333 2.99
0.5987129048989567 2.7
0.36684341946235766 2.96
0.5515747371737004 2.59
0.5918987762398348 2.84
0.5728597149611224 2.98
0.6012082449419962 2.56
0.38406230190695 2.9450000000000003
0.6115775420771656 2.575
0.37804737557801776 2.54
0.43140767756535764 2.62
0.38921744428376215 2.665
0.40077020754291054 2.6500000000000004
0.4178174157889174 2.79
0.4483852662320184 2.605
0.5312982350079939 2.87
0.46657410190012416 2.9000000000000004
0.5734354627583306 2.98
0.47806467912602424 0.43117547512160953 2.732.915

0.375093703668913 2.51
0.362540127722087 2.8200

In [20]:
print("MAX PCC: ", sorted_list[-1][0])
print("ALP:", sorted_list[-1][1])

MAX PCC:  0.6116038875278618
ALP: 1.0


In [ ]:
# # 单线程
# # 加载保存的数组
# lin = np.load("lin.npy")
# PCC_mean = 0
# res_pcc_list = []
# for alp in lin:
#     for t_coum in range(2, coum+1, 1):    #邻居
#         a = 1
#         Name_data = []
#         for linename in columns:
#             #求PCC
#             alpha = []
#             result_pcc = []
#             pcc_split = []
#             pre = []
#             for elem in range(39):
#                 pp = []
#                 for j in TTT[(coum * a - t_coum):(coum * a)]: 
#                     pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))#改函数
#                 pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
#             pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
#             result_pcc.append(np.max(pcc_split))
#             alpha.append(lin[pcc_split.index(np.max(pcc_split))])
#             pcc=result_pcc[0]
#             #求MSE
#             r1 = []
#             ct = 0
#             alp = alpha[0]
#             for elem in range(39):
#                 pp = []
#                 for j in TTT[(coum * a - t_coum):(coum * a)]:
#                     pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  # 改函数
#                 r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) -
#                                 dict_columns_split[f"{linename},{elem}"]['y_line'])
#             mse=np.mean(np.square(r1))
#             rmse=math.sqrt(mse)
#             Name_data.append([linename, t_coum, alp, mse, pcc, rmse])
#             a = a+1
#         pcc_mean = pd.DataFrame(Name_data, columns=['drug', 'coum', "alp", "mse", "pcc", "rmse"])["pcc"].mean()
#         if pcc_mean > PCC_mean:
#             PCC_mean = pcc_mean
#             res_pcc_list = Name_data
# # pcc_mean = pd.DataFrame(res_pcc_list, columns=['drug', 'coum', "alp", "mse", "pcc", "rmse"]).to_csv('pcc_unite.csv', header=None, index=False)

In [ ]:
pd.DataFrame(max_res, columns=['drug', 'coum', "alp", "mse", "pcc", "rmse"])